Photodiode Analysis Code
Ansley Kunnath
Updated 04/02/24
Run with Python 3.9.12

In [176]:
import mne
import numpy as np
from mne.preprocessing import (ICA)
from scipy.signal import find_peaks
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("TkAgg")
import scipy.stats as stats

eeg_path = "/Users/ansle/Documents/GitHub/undergrad_files/"  # You will need to change this location
file_name = "PhotoDiode_2024-03-22_10-24-46"
file_eeg = eeg_path + file_name + ".eeg"
file_vhdr = eeg_path + file_name + ".vhdr"
file_vmrk = eeg_path + file_name + ".vmrk"

# Load and plot the raw data
raw = mne.io.read_raw_brainvision(file_vhdr)
events, event_id = mne.events_from_annotations(raw)

#raw.crop(tmin=22, tmax=190)
#raw.plot()


Extracting parameters from /Users/ansle/Documents/GitHub/undergrad_files/PhotoDiode_2024-03-22_10-24-46.vhdr...
Setting channel info structure...
Used Annotations descriptions: ['Marker/Impedance', 'New Segment/', 'Stimulus/s1', 'Stimulus/s2', 'Stimulus/s3', 'Stimulus/s5']


/var/folders/dv/s4k5_3r51ml3q36bgbcdsgbr0000gn/T/ipykernel_45409/3637897116.py:16: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_brainvision(file_vhdr)


In [177]:
# Create epochs for checkerboard events

stimulus_s1_events = events[events[:, 2] == event_id['Stimulus/s1']]
stimulus_s1_events, event_id
tmin, tmax = 0, 0.250  # start and end time around each event in seconds
epochs = mne.Epochs(raw, events=stimulus_s1_events, event_id=event_id['Stimulus/s1'],
                    tmin=tmin, tmax=tmax, baseline=None, preload=True)
epochs = epochs.pick_channels(['BIP3'])


Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Loading data for 100 events and 126 original time points ...
0 bad epochs dropped


In [178]:
# Calculate the FIRST latency <10% average for each epoch --> 5.22 ms
peak_latencies = []
for epoch in epochs.get_data():
    mean_amplitude = np.mean(epoch)
    screen_changes = np.argmax(epoch < (-0.1 * mean_amplitude))
    if (epoch < (-0.1 * mean_amplitude)).any():
        first_time = epochs.times[screen_changes]
    else:
        first_time = None
    peak_latencies.append(first_time)

# Calculate latencies in milliseconds
peak_latencies_ms = np.array(peak_latencies) * 1000
average_latency_ms = np.mean(peak_latencies_ms)
total_events = len(peak_latencies_ms)

# Calculate latency CI in milliseconds
sem_latency = stats.sem(peak_latencies)  # SEM = std / sqrt(n)
confidence_level = 0.95
ci_width = sem_latency * stats.t.ppf((1 + confidence_level) / 2, len(peak_latencies_ms) - 1)
confidence_interval = (average_latency_ms - ci_width, average_latency_ms + ci_width)

print(f"Mean Latency: {average_latency_ms:.3f}")
print(f"95% Confidence Interval for the Latencies: {confidence_interval}")
print(f"Total Events: {total_events}")


Mean Latency: 5.220
95% Confidence Interval for the Latencies: (5.217806079200246, 5.222193920799754)
Total Events: 100


In [179]:
# Calculate the FIRST latency <40% average for each epoch --> 5.0 ms

peak_latencies = []
for epoch in epochs.get_data():
    mean_amplitude = np.mean(epoch)
    screen_changes = np.argmax(epoch < (-0.4 * mean_amplitude))
    if (epoch < (-0.4 * mean_amplitude)).any():
        first_time = epochs.times[screen_changes]
    else:
        first_time = None
    peak_latencies.append(first_time)

# Calculate latencies in milliseconds
peak_latencies_ms = np.array(peak_latencies) * 1000
average_latency_ms = np.mean(peak_latencies_ms)
total_events = len(peak_latencies_ms)

# Calculate latency CI in milliseconds
sem_latency = stats.sem(peak_latencies)  # SEM = std / sqrt(n)
confidence_level = 0.95
ci_width = sem_latency * stats.t.ppf((1 + confidence_level) / 2, len(peak_latencies_ms) - 1)
confidence_interval = (average_latency_ms - ci_width, average_latency_ms + ci_width)

print(f"Mean Latency: {average_latency_ms}")
print(f"95% Confidence Interval for the Latencies: {confidence_interval}")
print(f"Total Events: {total_events}")


Mean Latency: 5.0
95% Confidence Interval for the Latencies: (4.997765413288609, 5.002234586711391)
Total Events: 100


In [180]:
# Histogram of latency distributions

plt.figure(figsize=(10, 6))
plt.hist(peak_latencies_ms, bins=10, color='skyblue', edgecolor='black')
plt.axvline(average_latency_ms, color='red', linestyle='dashed', linewidth=2)
plt.title('Distribution of Latencies')
plt.xlabel('Latency (milliseconds)')
plt.ylabel('Frequency')
plt.legend(['Average Latency', 'Latencies'])
plt.grid(True)

plt.show()

In [181]:
import numpy as np
import matplotlib.pyplot as plt

# Scatter plot of the latencies
plt.figure(figsize=(10, 6))
plt.scatter(range(len(peak_latencies_ms)), peak_latencies_ms, color='black')
plt.axhline(average_latency_ms, color='red', linestyle='dashed', linewidth=2)
plt.title('Latencies')
plt.xlabel('Event')
plt.ylabel('Latency (milliseconds)')
plt.grid(True)

plt.show()

The rest of this code were previous attempts that don't work

In [ ]:
# Don't use this, it uses an abritary cut-off of 125e-6 and 
# averages the latencies of all the peaks instead of just the first one

import mne
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'raw' and 'event_id' are already defined and loaded

# Update the epoch parameters for the new time window and amplitude threshold
tmin_updated, tmax_updated = 0, 0.250
amplitude_threshold = 125e-6  # 125 µV in Volts
events, event_id = mne.events_from_annotations(raw)
stimulus_s1_events = events[events[:, 2] == event_id['Stimulus/s1']]

# Recreate epochs for Stimulus/s1 within the specified window
epochs_updated = mne.Epochs(raw, events=stimulus_s1_events, event_id=event_id['Stimulus/s1'],
                            tmin=tmin_updated, tmax=tmax_updated, preload=True, baseline=None)

# Pick the BIP3 channel
epochs_updated = epochs_updated.pick_channels(['BIP3'])

# Find the first timepoint exceeding 125 µV after the stimulus in each epoch
exceeding_threshold_latencies_updated = []
for epoch in epochs_updated.get_data():
    # Find the index where the amplitude first exceeds 125 µV
    exceeding_index_updated = np.where(epoch[0, :] > amplitude_threshold)[0]
    if exceeding_index_updated.size > 0:  # Check if there's at least one exceeding point
        first_exceeding_time_updated = epochs_updated.times[exceeding_index_updated[0]]
        exceeding_threshold_latencies_updated.append(first_exceeding_time_updated)
    else:
        exceeding_threshold_latencies_updated.append(None)

# Filter out None values for calculating average and visualization
valid_latencies_updated = [latency for latency in exceeding_threshold_latencies_updated if latency is not None]

# Calculate the average latency of valid latencies
average_latency_threshold_updated = np.mean(valid_latencies_updated) if valid_latencies_updated else 0

print(f"Number of valid latencies: {len(valid_latencies_updated)}, Average latency: {average_latency_threshold_updated}")

# Visualize the distribution of valid latencies
plt.figure(figsize=(10, 6))
plt.hist(valid_latencies_updated, bins=20, color='orange', edgecolor='black')
plt.axvline(average_latency_threshold_updated, color='blue', linestyle='dashed', linewidth=2)
plt.title('Distribution of Photodiode Peaks >125 uV (0, 0.250)')
plt.xlabel('Latency (seconds)')
plt.ylabel('Frequency')
plt.legend(['Average Latency', 'Latencies'])
plt.grid(True)

plt.show()


In [149]:
# The heat map looks weird, not sure why...

epochs_time_windowed = mne.Epochs(raw, events=stimulus_s1_events, event_id=event_id['Stimulus/s1'],
                                  tmin=0, tmax=0.250, preload=True, baseline=(None, None))

epochs_time_windowed.plot_image(picks=['BIP3'], sigma=1.0, cmap='viridis', 
                          vmin=-150, vmax=150)


Not setting metadata
100 matching events found
Setting baseline interval to [0.0, 0.25] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 100 events and 126 original time points ...
0 bad epochs dropped
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated


/var/folders/dv/s4k5_3r51ml3q36bgbcdsgbr0000gn/T/ipykernel_45409/3220100308.py:4: RuntimeWarning: Cannot find channel coordinates in the supplied Evokeds. Not showing channel locations.
  epochs_time_windowed.plot_image(picks=['BIP3'], sigma=1.0, cmap='viridis',


[<Figure size 640x480 with 3 Axes>]